#                                 TensorFlow 简介

In [35]:
import tensorflow as tf
a = tf.constant([1.0, 2.0], name='a')
b = tf.constant([2.0, 3.0], name='b')
result = a + b
print "Rs Tensor: ", result

with tf.Session() as sess:
    print "result: ", sess.run(result)

Rs Tensor:  Tensor("add_10:0", shape=(2,), dtype=float32)
result:  [3. 5.]


In [ ]:
same_graph = a.graph is tf.get_default_graph()

print "same graph :", same_graph

### TensorFlow 可通过tf.Graph 生成图

In [33]:
g1 = tf.Graph()
print g1
with g1.as_default():
    # 在计算图g1中定义变量“v”, 并设置初始值为0
    v = tf.get_variable("v", shape=[1], initializer=tf.zeros_initializer())
    print "Tensor v1: ", v
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        print "Value v1: ", sess.run(v)
        
print "\n", 100*'-', "\n"
g2 = tf.Graph()
with g2.as_default():
    # 在计算图g2中定义变量“v”, 并设置初始值为1
    v = tf.get_variable("v", shape=[1], initializer=tf.ones_initializer())
    print "Tensor v2: ", v
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        print "Value v2: ", sess.run(v)


# 在计算图g1中读取变量v的取值
with tf.Session(graph=g1) as sess:
    sess.run(tf.initialize_all_variables())
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))
        

# 在计算图g2中读取变量v的取值
with tf.Session(graph=g2) as sess:
    sess.run(tf.initialize_all_variables())
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

Tensor v1:  <tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
Value v1:  [0.]

---------------------------------------------------------------------------------------------------- 

Tensor v2:  <tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
Value v2:  [1.]
[0.]
[1.]


In [54]:
# 声明w1、w2两个变量,这里通过seed参数设置随机种子, 保证每次运行得到的结果是一样的;
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

x = tf.placeholder(tf.float32, shape=(None, 2), name="input")
a = tf.matmul(x, w1)  # model logic
y = tf.matmul(a, w2)  # 输出

with tf.Session() as sess:
    sess.run(w1.initializer)
    sess.run(w2.initializer)
    
    print "w1:", "\n", sess.run(w1)
    print "w2:", "\n", sess.run(w2)
    print "\n"
    print "Input:\n", sess.run(x, feed_dict={x: [[0.7, 0.9], [0.1, 0.4], [0.5, 0.8]]})
    print "Output:\n", sess.run(y, feed_dict={x: [[0.7, 0.9], [0.1, 0.4], [0.5, 0.8]]})

w1: 
[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
w2: 
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]


Input:
[[0.7 0.9]
 [0.1 0.4]
 [0.5 0.8]]
Output:
[[3.957578 ]
 [1.1537654]
 [3.1674924]]


#### 完整的圣经网络样例

In [66]:
import tensorflow as tf
from numpy.random import RandomState

batch_size = 8  # batch 大小

w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))  # firstly layer weights
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))    # secondly layer weights

x = tf.placeholder(tf.float32, shape=(None, 2), name="x-input")     # input x values
y_ = tf.placeholder(tf.float32, shape=(None, 1), name="y-input")    # input y values


a = tf.matmul(x, w1)    # hidden output
y = tf.matmul(a, w2)   # prediction y


loss = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))   # cross entropy
opt = tf.train.AdamOptimizer(0.001).minimize(loss)   # optimizer

import numpy as np
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = np.array([[int(x1 + x2 < 1)] for (x1, x2) in X])

print X.shape, Y.shape

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    # print sess.run(w1), sess.run(w2)
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        sess.run(opt, feed_dict={x: X[start: end], y_: Y[start: end]})
        if i % 1000 == 0:
            total_cross_entropy = sess.run(loss, feed_dict={x: X, y_: Y})
            print "After %d training steps, cross entropy on all data is %g" % (i, total_cross_entropy)

(128, 2) (128, 1)


After 0 training steps, cross entropy on all data is 0.0674925


After 1000 training steps, cross entropy on all data is 0.0163385


After 2000 training steps, cross entropy on all data is 0.00907547


After 3000 training steps, cross entropy on all data is 0.00714436


After 4000 training steps, cross entropy on all data is 0.00578471
